In [30]:
import pandas as pd
import altair as alt


df = pd.read_csv(r'/Users/adyaverma/Downloads/Coffe_sales (1).csv')


df['Date'] = pd.to_datetime(df['Date'])

sales_by_date = df.groupby(['Date', 'coffee_name']).agg({
    'money': 'sum',
    'coffee_name': 'count'  #
}).rename(columns={'money': 'total_sales', 'coffee_name': 'num_transactions'}).reset_index()


selection = alt.selection_point(fields=['coffee_name'], bind='legend')


line_chart = alt.Chart(sales_by_date).mark_line(point=True).encode(
    x=alt.X('Date:T', 
            title='Date',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('total_sales:Q', 
            title='Total Sales ($)',
            axis=alt.Axis(format='$,.0f')),
    color=alt.Color('coffee_name:N', 
                    title='Coffee Type',
                    scale=alt.Scale(scheme='category10')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    tooltip=[
        alt.Tooltip('Date:T', title='Date', format='%Y-%m-%d'),
        alt.Tooltip('coffee_name:N', title='Coffee Type'),
        alt.Tooltip('total_sales:Q', title='Sales', format='$,.2f'),
        alt.Tooltip('num_transactions:Q', title='Transactions')
    ]
).add_params(
    selection
).properties(
    width=700,
    height=400,
    title='Coffee Sales Over Time by Type'
)


line_chart.show()

#average instead of all 
#group the coffee types in a way that makes sense 
#would rather have one line with a clear trend
#bar plot maybe instead of pie plot : side by side box plot to see what is more stable for selling 
# make a storyline 

alt.Chart(...)

This visualization tracks individual coffee sales transactions across eight different beverage types over 15 months from March 2024 through March 2025, using a lollipop chart where each colored dot represents a single sale. Latte (gray) and Americano with Milk (orange) consistently generate the highest-value transactions, with some sales exceeding 300, while specialty drinks like Hot Chocolate and Cocoa typically stay under 150. A dramatic sales gap appears between August and September 2024, followed by recovery in October, and Americano (blue) shows notably increased activity in February-March 2025. The overlapping data points reveal that multiple coffee types generate sales simultaneously throughout most periods, with Cappuccino, Latte, and Americano with Milk maintaining the most consistent presence as core menu items. This granular view allows stakeholders to identify both typical purchasing patterns and exceptional high-value transactions across all product categories.

This visualization strategically employs marks and channels with point marks (dots) and line marks (stems) to create lollipops, using position on both axes as the primary quantitative channel—vertical position encodes sales amount (which humans perceive with high accuracy) while horizontal position encodes time sequentially. Color hue serves as the categorical channel to distinguish eight coffee types (blue for Americano, orange for Americano with Milk, green for Cappuccino, etc.), with sufficient perceptual separability maintained even when points overlap, and a comprehensive legend provides the decoding reference system. I chose lollipops over bar or line charts because the stem-and-dot structure maintains optimal data-ink ratio and reduces visual clutter with hundreds of overlapping points, while the Gestalt principle of connection links each value to its zero baseline to reinforce quantitative relationships. The customized axis extends from $0-$350 with gridlines for reference, prioritizing transparency and expressiveness by showing individual transactions rather than aggregated statistics, which enables stakeholders to understand sales variability, identify outliers, and recognize temporal patterns essential for inventory and staffing decisions.

In [31]:


daily_spending = df.groupby('Date')['money'].sum().reset_index()
daily_spending.columns = ['Date', 'total_spending']


brush = alt.selection_interval(encodings=['x'])

main_histogram = alt.Chart(daily_spending).mark_bar().encode(
    x=alt.X('Date:T',
            title='Date',
            axis=alt.Axis(format='%b %d, %Y', labelAngle=-45),
            scale=alt.Scale(domain=brush)),
    y=alt.Y('total_spending:Q',
            title='Total Spending ($)',
            axis=alt.Axis(format='$,.0f')),
    tooltip=[
        alt.Tooltip('Date:T', title='Date', format='%Y-%m-%d'),
        alt.Tooltip('total_spending:Q', title='Total Spending', format='$,.2f')
    ]
).properties(
    width=800,
    height=400,
    title='Daily Coffee Spending Over Time'
)


overview = alt.Chart(daily_spending).mark_bar().encode(
    x=alt.X('Date:T',
            title='Select a date range to zoom',
            axis=alt.Axis(format='%b %Y')),
    y=alt.Y('total_spending:Q',
            title='Spending ($)',
            axis=alt.Axis(format='$,.0f')),
).add_params(
    brush
).properties(
    width=800,
    height=60,
    title='Overview - Click and drag to zoom'
)


combined_chart = alt.vconcat(main_histogram, overview).resolve_scale(
    y='independent'
)


combined_chart.show()

alt.VConcatChart(...)

This visualization displays daily total coffee spending over a 15-month period from March 2024 through March 2025, with each bar representing the aggregate sales for a single day. The data reveals significant day-to-day variability in spending, with daily totals ranging from approximately 100 on slower days to peaks exceeding $800 in mid-October 2024 and late February 2025. A clear upward trend emerges in the fall months (September-November 2024), where sustained higher spending levels suggest increased customer traffic or higher-value purchases, followed by a decline in December 2024 and January 2025, likely reflecting seasonal patterns or holiday closures. The interactive overview panel below the main chart provides context by showing the entire dataset at a compressed scale, allowing users to click and drag to zoom into specific time periods for detailed analysis. This dual-view design enables both macro-level pattern recognition (identifying seasonal trends and growth periods) and micro-level investigation (examining specific days or weeks of interest).

This visualization employs a bar chart with area marks to represent daily spending totals, using vertical position as the primary quantitative channel encoding dollar amounts and horizontal position encoding time sequentially across the x-axis. The color channel uses a single blue hue with subtle gradient shading to create visual cohesion while the varying bar heights leverage length as an anditional quantitative channel, which humans perceive accurately for magnitude comparisons. I customized the layout to include an interactive overview + detail design pattern with two coordinated views: the main chart shows the full resolution data with gridlines for precise value reading, while the compressed overview panel below enables navigation through the dataset via click-and-drag zooming, addressing the challenge of displaying 450+ days of data in a readable format. The bar chart format was chosen over line charts or point-based visualizations because it emphasizes the discrete nature of daily totals and uses the Gestalt principle of enclosure to clearly show each day's contribution to overall revenue, with bars extending from a zero baseline to maintain accurate magnitude perception and avoid misleading visual comparisons. This design prioritizes both effectiveness (accurate quantitative comparison) and interactivity (user-controlled exploration), enabling stakeholders to identify revenue trends, investigate anomalies like the October peak, and make data-driven decisions about staffing, inventory, and business strategy. 

In [48]:
filter1 = df['coffee_name'] == 'Cocoa'
filter2 = df['coffee_name'] == 'Americano'
filter3 = df['coffee_name'] = 'Latte'

df_filtered = df[filter1 | filter2 | filter3]


TypeError: Cannot perform 'or_' with a dtyped [bool] array and scalar of type [bool]

In [49]:
df_filtered

,hour_of_day,cash_type,money,coffee_name,Time_of_Day,Weekday,Month_name,Weekdaysort,Monthsort,Date,Time
3,13,card,28.90,Americano,Afternoon,Fri,Mar,5,3,2024-03-01,13:46:33.006000
8,19,card,38.70,Cocoa,Night,Fri,Mar,5,3,2024-03-01,19:22:01.762000
11,10,card,28.90,Americano,Morning,Sat,Mar,6,3,2024-03-02,10:22:06.957000
14,14,card,28.90,Americano,Afternoon,Sat,Mar,6,3,2024-03-02,14:38:35.535000
16,17,card,28.90,Americano,Night,Sat,Mar,6,3,2024-03-02,17:34:54.969000
...,...,...,...,...,...,...,...,...,...,...,...
3534,12,card,25.96,Americano,Afternoon,Sat,Mar,6,3,2025-03-22,12:25:59.676000
3535,13,card,25.96,Americano,Afternoon,Sat,Mar,6,3,2025-03-22,13:06:21.590000
3543,14,card,35.76,Cocoa,Afternoon,Sun,Mar,7,3,2025-03-23,14:43:37.362000
3544,14,card,35.76,Cocoa,Afternoon,Sun,Mar,7,3,2025-03-23,14:44:16.864000
